In [56]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from keys import keys
import models  # flask-sqlacodegen

In [57]:
user = ''
password = keys.get("postgres", "./keys")
aws_rds_endpoint = ''
db = ''
engine = create_engine(f'postgresql://{user}:{password}@{aws_rds_endpoint}/{db}')
Session = sessionmaker(bind=engine)
session = Session()

In [65]:
def update_pog(company_id, store_id, device_id, df, insert_mode=False):
    if session.query(models.Company).filter_by(company_id=company_id).first() is None:
        print('company_id가 존재하지 않습니다.')
        return False
    if session.query(models.Store).filter_by(store_id=store_id).first() is None:
        print('store_id가 존재하지 않습니다.')
        return False
    try:
        if insert_mode:
            # insert devices
            if session.query(models.Device).filter_by(device_id=device_id).first() is None:
                print('device_id가 존재하지 않습니다. device 정보를 추가합니다.')
                if device_id[0] == 's':
                    device_install_type = 'S'
                elif device_id[0] == 'w':
                    device_install_type = 'W'
                session.add(models.Device(device_id=device_id, store_id=store_id, device_install_type=device_install_type, operation=False))
                session.commit()
            
            # get_device_pkey
            device_pkey = session.query(models.Device.device_pkey)\
                .select_from(models.Device)\
                .join(models.Store, models.Store.store_id == models.Device.store_id)\
                .join(models.Company, models.Company.company_id == models.Store.company_id)\
                .filter(models.Company.company_id == company_id,
                        models.Store.store_id == store_id,
                        models.Device.device_id == device_id)\
                .first().device_pkey

            # insert shelves
            for shelf_floor in set(df.shelf_floor):
                if session.query(models.Shelf).filter_by(device_pkey=device_pkey, shelf_floor=shelf_floor).first() is None:
                    print('shelf_floor가 존재하지 않습니다. shelf 정보를 추가합니다.')
                    sub_df = df[df[shelf_floor] == shelf_floor]
                    shelf_storage_type = sub_df['shelf_storage_type'][0]
                    shelf_model = sub_df['shelf_model'][0]
                    session.add(device_pkey=device_pkey, shelf_floor=shelf_floor, shelf_storage_type=shelf_storage_type, shelf_model=shelf_model)
                    session.commit()

            for d in df.itertuples():
                # get shelf_pkey
                shelf_pkey = session.query(models.Shelf.shelf_pkey)\
                    .select_from(models.Shelf)\
                    .join(models.Device, models.Device.device_pkey == models.Shelf.device_pkey)\
                    .join(models.Store, models.Store.store_id == models.Device.store_id)\
                    .join(models.Company, models.Company.company_id == models.Store.company_id)\
                    .filter(models.Company.company_id == company_id,
                            models.Store.store_id == store_id,
                            models.Device.device_id == device_id,
                            models.Shelf.shelf_floor == d.shelf_floor)\
                    .first().shelf_pkey

                # get design_pkey
                design_pkey = session.query(models.Design.design_pkey).filter_by(goods_id=d.goods_id).all()[-1].design_pkey
                
                # get cells
                cells = session.query(models.Cell).filter_by(shelf_pkey=shelf_pkey).first()

                if cells is None:
                    # insert cells
                    session.add(shelf_pkey=shelf_pkey,
                                design_pkey_master=design_pkey,
                                cell_column=d.cell_column,
                                stock_count_max=d.stock_count_max,
                                mode=d.mode)
                else:
                    # update cells
                    cells.design_pkey_master = design_pkey
                    cells.stock_count_max = d.stock_count_max
                    cells.mode = d.mode
            session.commit()

            for d in df.itertuples():
                shelf_pkey = session.query(models.Shelf.shelf_pkey)\
                    .select_from(models.Shelf)\
                    .join(models.Device, models.Device.device_pkey == models.Shelf.device_pkey)\
                    .join(models.Store, models.Store.store_id == models.Device.store_id)\
                    .join(models.Company, models.Company.company_id == models.Store.company_id)\
                    .filter(models.Company.company_id == company_id,
                            models.Store.store_id == store_id,
                            models.Device.device_id == device_id,
                            models.Shelf.shelf_floor == d.shelf_floor)\
                    .first().shelf_pkey

                cell_pkey = session.query(models.Cell.cell_pkey).filter_by(shelf_pkey=shelf_pkey, cell_column=d.cell_column).first().cell_pkey

                # get loadcells
                loadcells = session.query(models.Loadcell).filter_by(shelf_pkey=shelf_pkey, loadcell_column=d.loadcell_column).first()

                if loadcells is None:
                    # insert loadcell
                    session.add(models.Loadcell(shelf_pkey=shelf_pkey, cell_pkey=cell_pkey, loadcell_column=d.loadcell_column))
                else:
                    # update loadcell
                    loadcells.cell_pkey = cell_pkey
            session.commit()

        else:
            new_cells = pd.DataFrame({''})
            for d in df.itertuples():
                # get shelf_pkey
                shelf_pkey = session.query(models.Shelf.shelf_pkey)\
                    .select_from(models.Shelf)\
                    .join(models.Device, models.Device.device_pkey == models.Shelf.device_pkey)\
                    .join(models.Store, models.Store.store_id == models.Device.store_id)\
                    .join(models.Company, models.Company.company_id == models.Store.company_id)\
                    .filter(models.Company.company_id == company_id,
                            models.Store.store_id == store_id,
                            models.Device.device_id == device_id,
                            models.Shelf.shelf_floor == d.shelf_floor)\
                    .first().shelf_pkey

                # get design_pkey
                design_pkey = session.query(models.Design.design_pkey).filter_by(goods_id=d.goods_id).all()[-1].design_pkey
                
                # get cell
                cell = session.query(models.Cell).filter_by(shelf_pkey=shelf_pkey, cell_column=d.cell_column).first()

                # update cells
                cell.design_pkey_master = design_pkey
                cell.stock_count_max = d.stock_count_max
                cell.mode = d.mode

                # get loadcell
                loadcell = session.query(models.Loadcell).filter_by(shelf_pkey=shelf_pkey, loadcell_column=d.loadcell_column).first()

                # update loadcell
                loadcell.cell_pkey = cell.cell_pkey
            session.commit()

    except Exception as e:
        session.rollback()
        raise e

In [68]:
df = pd.read_csv('CU_역삼점_POG_test.csv', dtype={'shelf_floor': 'int', 'cell_column': 'int', 'loadcell_column': 'int', 'goods_id': 'str',
                                                 'stock_count_max': 'int', 'mode': 'str', 'shelf_storage_type': 'str', 'shelf_model': 'str'})
df.head()

,shelf_floor,cell_column,loadcell_column,goods_id,stock_count_max,mode,shelf_storage_type,shelf_model
0,0,0,0,8801007071046,8,mix,CD,30944_s_bev
1,0,1,1,8802255808019,8,mix,CD,30944_s_bev
2,0,2,2,8806002007298,8,mix,CD,30944_s_bev
3,0,3,3,8801033000119,8,mix,CD,30944_s_bev
4,0,4,4,8801033000126,8,mix,CD,30944_s_bev


In [67]:
update_pog('0004', '30944', 's_00001', df, insert_mode=True)